In [2]:
# main.py
import numpy as np
from IPython.display import Audio, display, Markdown
import os

# Refactored module imports
from utils import load_data
from synthesis import synthesize_fm_chain, sine_wave
from objectives import prepare_target, compute_fft
from optimization import define_objective_function, run_optimization
from plotting import plot_time_domain_signal, plot_frequency_domain_signal, plot_error_history
from generate_wave_file import save_wave_file

# --- 1. Configuration ---
# General
FILE_PATH = 'tsv/cello_single.tsv'
DURATION = 1  # seconds
SAMPLE_RATE = 44100  # samples per second

# Synthesis & Optimization
N_OSCILLATORS = 4
OBJECTIVE_TYPE = 'mfcc_distance'  # 'mfcc_distance', 'euclidean_distance', etc.
OPTIMIZATION_METHOD = 'differential_evolution'
MAX_ITERATIONS = 1000

# --- 2. Load Data ---
frequencies, amplitudes = load_data(FILE_PATH)

# --- 3. Generate and Render Original Target Sound (Additive Synthesis) ---
display(Markdown("### 1. Original Target Sound (from TSV)"))
print("Generating audio from target partials for comparison...")

# Create the signal by summing sine waves
target_signal = np.zeros(int(SAMPLE_RATE * DURATION))
for freq, amp in zip(frequencies, amplitudes):
    target_signal += sine_wave(freq, amp, DURATION, SAMPLE_RATE)

# Normalize the target signal before saving
max_val = np.max(np.abs(target_signal))
if max_val > 0:
    target_signal /= max_val

# Save and display the target audio
target_audio_path = save_wave_file(
    signal=target_signal,
    source_sr=SAMPLE_RATE,
    filename="target_additive_cello.wav"
)
display(Audio(filename=target_audio_path))

# --- 4. Prepare Data for Optimization ---
target_data = prepare_target(frequencies, amplitudes, DURATION, SAMPLE_RATE, OBJECTIVE_TYPE)

# --- 5. Define Optimization Problem ---
bounds = []
for i in range(N_OSCILLATORS):
    bounds.append((1, 5000))  # Frequency bounds
    bounds.append((0, 12))   # Amplitude bounds (modulation index)

objective_function = define_objective_function(OBJECTIVE_TYPE, target_data, DURATION, SAMPLE_RATE)

# --- 6. Run Optimization ---
display(Markdown("### 2. Running Optimization"))
print(f"Running {OPTIMIZATION_METHOD} for {MAX_ITERATIONS} iterations...")
result, error_history = run_optimization(
    OPTIMIZATION_METHOD,
    bounds,
    objective_function,
    MAX_ITERATIONS
)

# --- 7. Process and Analyze Results ---
display(Markdown("### 3. Optimization Results"))
print("\nOptimization Complete.")
print(f"Final Objective Value: {result.fun}")

optimal_params = result.x
optimal_frequencies = optimal_params[0::2]
optimal_amplitudes = optimal_params[1::2]

# Generate the final, optimized signal using the single source of truth
optimized_signal = synthesize_fm_chain(optimal_params, DURATION, SAMPLE_RATE)
fft_magnitude, fft_freqs = compute_fft(optimized_signal, SAMPLE_RATE)

# Plotting
plot_time_domain_signal(optimized_signal, SAMPLE_RATE)
plot_frequency_domain_signal(fft_freqs, fft_magnitude, frequencies, amplitudes)
plot_error_history(error_history)

# Print optimal parameters
print("\nOptimal Parameters:")
for i in range(N_OSCILLATORS):
    # --- THIS IS THE KEY CHANGE ---
    # The Carrier is now the FIRST oscillator in the list (index 0)
    # The last modulator in the chain is Oscillator N (index N-1)
    if i == 0:
        role = "Carrier"
    else:
        # Osc 2 modulates Osc 1, Osc 3 modulates Osc 2, etc.
        role = f"Modulator for Osc {i}"

    print(f"  Oscillator {i+1} ({role}):")
    print(f"    Frequency: {optimal_frequencies[i]:.2f} Hz")
    print(f"    Amplitude: {optimal_amplitudes[i]:.2f}\n")


# --- 8. Synthesize and Display Optimized Sound ---
display(Markdown("### 4. Optimized Sound (FM Synthesis)"))

# Save and display the optimized audio ONCE
output_filename = f'optimized_fm_cello_{OPTIMIZATION_METHOD}_{OBJECTIVE_TYPE}.wav'
optimized_audio_path = save_wave_file(
    signal=optimized_signal,
    source_sr=SAMPLE_RATE,
    target_sr=44100,
    bit_depth=16,
    filename=output_filename
)
display(Audio(filename=optimized_audio_path))

### 1. Original Target Sound (from TSV)

Generating audio from target partials for comparison...
[2025-06-27 22:39:36] File saved successfully to: rendered_audio\target_additive_cello.wav


Generating and analyzing target signal...


### 2. Running Optimization

Running differential_evolution for 1000 iterations...


  4%|▍         | 38/1000 [00:27<11:05,  1.45iter/s]

KeyboardInterrupt: 